# Intended to preprocess and upload SSkin model to GCS

## Requirement
- Kaggle credentials (in current directory)
- GCP project Service Account's keys with permission to upload objects to bucket (also stored in current directory)


In [15]:
! rm -rf w-preprocess

In [ ]:
! pip install kaggle

In [ ]:
# buat direktori run
! mkdir raw-models

In [7]:
# run ini sebelum ambil data dari kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ambil data dari kaggle (change this kaggle's username/notebook to ur own username/notebook)
! kaggle kernels output amarrahmankamal/notebook4d22c1b5ec -p raw-models/

In [9]:
! mv raw-models/saved_model raw-models/model1

In [10]:
from google.cloud import storage
import tensorflow as tf
import datetime
import glob
import os

In [11]:
# ganti nama modelnya setiap kali run
model_folder_name = 'my_model'

# load model folder from current directory
raw_models_path = 'raw-models'
model = tf.saved_model.load(raw_models_path + '/' + model_folder_name)

In [ ]:
# set path unik setiap kali run (sekali run, 1 path unik)
model_build_datetime = str(datetime.datetime.now())
final_local_model_path = 'w-preprocess/model-' + model_build_datetime

# Tambahin layer preprocess input decode base64, karena Vertex AI endpoint expect input dalam bentuk base64
IMAGE_SIZE = 224

def _preprocess(bytes_inputs):
    decoded = tf.io.decode_jpeg(bytes_inputs, channels=3)
    resized = tf.image.resize(decoded, size=(IMAGE_SIZE, IMAGE_SIZE))
    return tf.cast(resized, dtype=tf.float32)

def _get_serve_image_fn(model):
    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def serve_image_fn(bytes_inputs):
        decoded_images = tf.map_fn(_preprocess, bytes_inputs, dtype=tf.float32)
        return model(decoded_images)
    return serve_image_fn

signatures = {
    "serving_default": _get_serve_image_fn(model).get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string)
    )
}

tf.saved_model.save(model, final_local_model_path, signatures=signatures)

In [18]:
# upload model yang udah dibuat ke Google Cloud Storage (butuh credentials.json)

# !PERHATIAN! butuh credentials.json sebelum cell ini di run
credentials_path = 'credentials.json'
client = storage.Client.from_service_account_json(credentials_path)

# Untuk tiap project nanti gantinya di project_name aja
project_name = 'testing-purpose'

bucket_name = project_name + '-models'
bucket_path = 'models/sskin-model-' + model_build_datetime

bucket = client.bucket(bucket_name)

def upload_local_directory_to_gcs(local_path, bucket, gcs_path):
    assert os.path.isdir(local_path)
    for local_file in glob.glob(local_path + '/**'):
        if not os.path.isfile(local_file):
           upload_local_directory_to_gcs(local_file, bucket, gcs_path + "/" + os.path.basename(local_file))
        else:
           remote_path = os.path.join(gcs_path, local_file[1 + len(local_path):])
           blob = bucket.blob(remote_path)
           blob.upload_from_filename(local_file)

# di comment biar nggak auto upload, kalo mau upload uncomment line dibawah dan run cell ini doang

upload_local_directory_to_gcs(final_local_model_path, bucket, bucket_path)